# Seq2seq NMT with RNN



[Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)

**NOTE:**

-  use clean bpe data
-  use a piece of triaing data during coding or low in credits

You have to implement:

- Encoder
- Attention (Bahdanau)
- training loop
- extra: BLEU model selection

Goal:

- Loss in training, validation and test





In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import random
import time

In [ ]:
#if you dont have bpe data use sacremoese tokenizer
#!pip install sacremoses

In [ ]:
#which libraries are we using!!?
!pip freeze > requirements.txt

In [ ]:
!cat requirements.txt

In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
import io

#0=en 1=de
# soure and target data
#NOTE: USE clean bpe data!

train_filepaths = ['train.100k.en-de.bpe.en', 'train.100k.en-de.bpe.de']
val_filepaths = ['dev.500.en-de.bpe.en', 'dev.500.en-de.bpe.de']
test_filepaths = ['test.500.en-de.bpe.en', 'test.500.en-de.bpe.en']


#TODO use clean bpe tokenized data!!!
#de_tokenizer = get_tokenizer('moses', language='de')
de_tokenizer = None
#en_tokenizer = get_tokenizer('moses', language='en')
en_tokenizer = None

def build_vocab(filepath, tokenizer=None):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for string_ in f:
      #counter.update(tokenizer(string_))
      counter.update(string_.split())
  return vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

#Vocab
en_vocab = build_vocab(train_filepaths[0], en_tokenizer)
de_vocab = build_vocab(train_filepaths[1], de_tokenizer)

en_vocab.set_default_index(en_vocab['<unk>'])
de_vocab.set_default_index(de_vocab['<unk>'])

def data_process(filepaths):
  raw_en_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_de_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_en, raw_de) in zip(raw_en_iter, raw_de_iter):
    en_tensor_ = torch.tensor([en_vocab[token] for token in raw_en.split()], #en_tokenizer(raw_en)
                            dtype=torch.long)
    de_tensor_ = torch.tensor([de_vocab[token] for token in raw_de.split()], #de_tokenizer(raw_de)
                            dtype=torch.long)
    data.append((en_tensor_, de_tensor_))
  return data

#pre-process
train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)




In [ ]:
#NOTE: if you are low on credits or testing only use a piece of the data eg 20K segments
train_data = train_data[:10000]

In [ ]:
len(train_data)

1000

Define the device.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


Create the iterators.

In [ ]:
BATCH_SIZE = 8
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
    en_batch, de_batch = [], []
    for (en_item, de_item) in data_batch:
        de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
    de_batch = pad_sequence(de_batch, padding_value=PAD_IDX, batch_first=True)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX, batch_first=True)
    return en_batch, de_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=False, collate_fn=generate_batch)


## Building the Seq2Seq Model

### Encoder




In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = #[YOUR CODE] GRU(embeding size, encoder hidden size) NOTE: bidirectional batch_first

        self.fc = #[YOUR CODE] linear(encoder hidden size * 2, decoder hidden size)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):


        #[B, S]

        embedded = self.embedding(src)

        embedded = self.dropout(embedded)


        #[B, S, E]

        outputs, hidden = #[YOUR CODE] rrn(embeddings)


        #[B, S, H*2]

        #h[n layers * num directions, batch size, hid dim]

        #[forward_1, backward_1, forward_2, backward_2, ...]
        #

        #[-2, :, : ] last state forward RNN
        #[-1, :, : ] last state backward RNN
        #print('hid', hidden.size())
        h1 = #[YOUR CODE] last state forward RNN
        h2 = #[YOUR CODE] last state backward RNN

        #https://pytorch.org/docs/main/generated/torch.cat.html
        h_cat = #[YOUR CODE] concatenate h1 amd h2 on seq dim

        hidden = #[YOUR CODE] tanh(linear(hidden_cat))

        #[B, S, H*2]

        return outputs, hidden

# Attention

## Luong Attention




In [ ]:
class LuongAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)

    def forward(self, hidden, encoder_outputs): #keys, query

        #[batch size, dec hid dim]
        #[src len, batch size, enc hid dim * 2]

        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]

        #x times decoder hidden state for src_len
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        #[batch size, src len, dec hid dim]

        scores = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2)))


        #[batch size, src len, dec hid dim]

        attention = self.v(scores).squeeze(2)

        #[batch size, src len]

        return F.softmax(attention, dim=1)

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(dec_hid_dim, dec_hid_dim, bias=False)
        self.Ua = nn.Linear(enc_hid_dim * 2, dec_hid_dim, bias=False)
        self.Va = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs): #keys, query

        batch_size = #[YOUR CODE]
        src_len = #[YOUR CODE]

        #x times decoder hidden state for src_len
        hidden = #[YOUR CODE]

        scores = #[YOUR CODE] Va(tanh(Wa(hidden) + Ua(encoder outputs)))
        scores = scores.squeeze(2)

        weights = #[YOUR CODE] softmax(scores, dim seq)

        return weights

### Decoder



In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim

        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim, batch_first=True)

        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

        #attention
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)

    def forward(self, input, hidden, encoder_outputs):

        #[batch size]
        #[batch size, dec hid dim]
        #[src len, batch size, enc hid dim * 2]


        input = input.unsqueeze(0)

        #[1, batch size]

        embedded = self.dropout(self.embedding(input))

        #[1, batch size, emb dim]

        a = self.attention(hidden, encoder_outputs)

        #[batch size, src len]

        a = a.unsqueeze(1)

        #[batch size, 1, src len]

        #[batch size, src len, enc hid dim * 2]

        weighted = torch.bmm(a, encoder_outputs)

        #[batch size, 1, enc hid dim * 2]

        weighted = weighted.permute(1, 0, 2)

        #[1, batch size, enc hid dim * 2]

        rnn_input = torch.cat((embedded, weighted), dim = 2)

        #[1, batch size, (enc hid dim * 2) + emb dim]
        #[B, 1, (enc hid dim * 2) + emb dim]

        rnn_input = rnn_input.permute(1, 0, 2)
        hidden = hidden.unsqueeze(0)

        output, hidden = self.rnn(rnn_input, hidden)

        #[seq len, batch size, dec hid dim * n directions]
        #[n layers * n directions, batch size, dec hid dim]

        #[1, batch size, dec hid dim]
        #[1, batch size, dec hid dim]

        embedded = embedded.squeeze(0)
        output = output.squeeze(1)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))

        #[batch size, output dim]

        return prediction, hidden.squeeze(0)

### Seq2Seq




In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        #[src len, batch size]
        #[trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        # 0.75 teacher forcing 75% of the time

        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim


        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        #encoder_outputs is all hidden states of the input sequence
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)

        #first input to the decoder is the <sos> tokens
        input = trg[:,0]
        # unroll RNN
        for t in range(1, trg_len):

            #insert input token embedding, previous hidden state and all encoder hidden states

            output, hidden = self.decoder(input, hidden, encoder_outputs)

            #predictions
            outputs[:, t] = output

            #teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio

            #greedy search
            top1 = output.argmax(1)

            #if teacher forcing, use gold token as next input
            #if not, use predicted token
            input = trg[:, t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model



In [ ]:
INPUT_DIM = len(en_vocab)
OUTPUT_DIM = len(de_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.2

attn = LuongAttention(ENC_HID_DIM, DEC_HID_DIM)
#[YOUR CODE] Bahdanau att
#attn = BahdanauAttention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
print(len(en_vocab)) #BPE size 16k approx
print(len(de_vocab))

5542
6700


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5542, 256)
    (rnn): GRU(256, 512, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): LuongAttention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(6700, 256)
    (rnn): GRU(1280, 512, batch_first=True)
    (fc_out): Linear(in_features=1792, out_features=6700, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (attn): Linear(in_features=1536, out_features=512, bias=True)
    (v): Linear(in_features=512, out_features=1, bias=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 22,367,788 trainable parameters


We create an optimizer.

In [ ]:
optimizer = #[YOUR CODE] Adam lr 1e-3

We initialize the loss function.

In [ ]:
TRG_PAD_IDX = de_vocab['<pad>'] #TODO
#https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
criterion = #[YOUR CODE] loss add ignore_index idx

In [ ]:
print(TRG_PAD_IDX)

1


In [ ]:
def train(model, iterator, optimizer, criterion, clip):

    #[YOUR CODE] set model train model.train()

    epoch_loss = 0

    for (src, trg) in tqdm(iterator):

        src, trg = #[YOUR CODE] to gpu

        #[YOUR CODE] optimizer zero grad

        output = #[YOUR CODE] model()

        #[trg len, batch size]
        #[trg len, batch size, output dim]

        output = output.permute(1, 0, 2)

        output_dim = output.shape[-1]
        trg = trg.permute(1, 0)

        output = output[1:].reshape(-1, output_dim)
        trg = trg[1:].reshape(-1)

        #[(trg len - 1) * batch size]
        #[(trg len - 1) * batch size, output dim]

        loss = #[YOUR CODE] criterion()

        #[YOUR CODE] loss backward

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        #[YOUR CODE] optimizer step

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):

    #[YOUR CODE] set model test/eval  model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for (src, trg) in iterator:
            src, trg = #[YOUR CODE] to device

            output = #[YOUR CODE] model() turn off teacher forcing 0

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output = output.permute(1, 0, 2)
            output_dim = output.shape[-1]
            trg = trg.permute(1, 0)
            output = output[1:].reshape(-1, output_dim)
            trg = trg[1:].reshape(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = #[YOUR CODE] criterion()

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    if valid_loss < best_valid_loss:
      #[YOUR CODE] extra add BLEU model selection
      best_valid_loss = valid_loss
      #torch.save(model.state_dict(), 'model.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}\tTrain PPL: {np.exp(train_loss):7.3f}')
    print(f'\t Validation Loss: {valid_loss:.3f}\tValidation PPL: {np.exp(valid_loss):7.3f}')

100%|██████████| 125/125 [00:18<00:00,  6.78it/s]


Epoch: 01
	Train Loss: 7.785	Train PPL: 2405.277
	 Validation Loss: 6.986	Validation PPL: 1081.010


100%|██████████| 125/125 [00:16<00:00,  7.43it/s]


Epoch: 02
	Train Loss: 6.955	Train PPL: 1048.880
	 Validation Loss: 6.758	Validation PPL: 860.717


100%|██████████| 125/125 [00:16<00:00,  7.55it/s]


Epoch: 03
	Train Loss: 6.646	Train PPL: 769.410
	 Validation Loss: 6.482	Validation PPL: 653.374


100%|██████████| 125/125 [00:16<00:00,  7.60it/s]


Epoch: 04
	Train Loss: 6.281	Train PPL: 534.420
	 Validation Loss: 6.145	Validation PPL: 466.177


100%|██████████| 125/125 [00:16<00:00,  7.40it/s]


Epoch: 05
	Train Loss: 5.913	Train PPL: 369.934
	 Validation Loss: 5.773	Validation PPL: 321.486


In [ ]:
#NOTE: load model from file
#model.load_state_dict(torch.load('model.pt'))

test_loss = evaluate(model, test_iter, criterion)

print(f'\tTest Loss: {test_loss:.3f}\tTest PPL: {np.exp(test_loss):7.3f}')

In [ ]:
#clean mem
del model
del train_iter
del valid_iter
del test_iter
torch.cuda.empty_cache()